In [1]:
from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row,SQLContext
from pyspark.sql import HiveContext

import sys
import requests

In [2]:
sc.setLogLevel("ERROR")

In [3]:
hive_context = HiveContext(sc)
ssc = StreamingContext(sc, 2)
ssc.checkpoint("checkpoint_TwitterApp")
dataStream = ssc.socketTextStream("localhost",9009)

In [4]:
country_lines = sc.textFile('file:///home/cloudera/Downloads/country-list.csv')

In [5]:
country_words = country_lines.map(lambda line: line.split(","))
list = country_words.collect()
country_tuples = country_words.map(lambda word:(word))
countryDF = sqlContext.createDataFrame(country_tuples, ["country", "code"])

AttributeError: type object 'list' has no attribute 'take'

In [6]:
def aggregate_tags_count(new_values, total_sum):
    return sum(new_values) + (total_sum or 0)

In [7]:
def send_df_to_dashboard(df):
    # extract the hashtags from dataframe and convert them into array
    top_tags = [str(t.hashtag) for t in df.select("hashtag").collect()]
    # extract the counts from dataframe and convert them into array
    tags_count = [p.hashtag_count for p in df.select("hashtag_count").collect()]
    # initialize and send the data through REST API
    url = 'http://localhost:5001/updateData'
    request_data = {'label': str(top_tags), 'data': str(tags_count)}
    response = requests.post(url, data=request_data)

In [8]:
def get_sql_context_instance(spark_context):
    if ('sqlContextSingletonInstance' not in globals()):
        globals()['sqlContextSingletonInstance'] = SQLContext(spark_context)
    return globals()['sqlContextSingletonInstance']

from pyspark.sql.functions import col 

def process_rdd(time, rdd):
    print("----------- %s -----------" % str(time))
    try:
        #type(rdd)
        # Get spark sql singleton context from the current context
        sql_context = get_sql_context_instance(rdd.context)
        # convert the RDD to Row RDD
        row_rdd = rdd.map(lambda w: Row(hashtag=w[0], hashtag_count=w[1]))
        # create a DF from the Row RDD
        hashtags_df = sql_context.createDataFrame(row_rdd)
        #hashtags_df = sqlContext.createDataFrame(row_rdd, ["hashtag", "hashtag_count"])
        joinedDF = countryDF.alias('c').join(hashtags_df.alias('t'),col('c.country') == col('t.hashtag')).select(col('c.code'), col('c.country'), col('t.hashtag_count'))
        #Register the dataframe as table
        joinedDF.registerTempTable("hashtags")
        #hashtags_df.registerTempTable("hashtags")
        # get the top 10 hashtags from the table using SQL and print them
        hashtag_counts_df = sql_context.sql("select hashtag, hashtag_count from hashtags order by hashtag_count desc limit 10")
        hashtag_counts_df.show()
        # call this method to prepare top 10 hashtags DF and send them
        send_df_to_dashboard(hashtag_counts_df)
    except:
        e = sys.exc_info()[0]
        print("Error: %s" % e)

In [9]:
# split each tweet into words
words = dataStream.flatMap(lambda line: line.split(" "))
# filter the words to get only hashtags, then map each hashtag to be a pair of (hashtag,1)
#x = words.filter(lambda x: x in list)
#hashtags = words.filter(lambda w: "#" in w).map(lambda x: (x, 1))
hashtags = words.map(lambda x: (x, 1))
# adding the count of each hashtag to its last count
tags_totals = hashtags.updateStateByKey(aggregate_tags_count)
# do processing for each RDD generated in each interval
tags_totals.foreachRDD(process_rdd)
#type(words)
#type(x)

In [ ]:
# start the streaming computation
ssc.start()
# wait for the streaming to finish
ssc.awaitTermination()

----------- 2018-05-22 09:22:34 -----------
Error: <class 'ValueError'>
----------- 2018-05-22 09:22:36 -----------
Error: <class 'ValueError'>
----------- 2018-05-22 09:22:38 -----------
Error: <class 'ValueError'>
----------- 2018-05-22 09:22:40 -----------
Error: <class 'ValueError'>
----------- 2018-05-22 09:22:42 -----------
Error: <class 'ValueError'>
----------- 2018-05-22 09:22:44 -----------
+----------+-------------+
|   hashtag|hashtag_count|
+----------+-------------+
| #judibola|            1|
|  #nigeria|            1|
|#agenbola…|            1|
|    #worl…|            1|
|   #brazil|            1|
|#argentina|            1|
+----------+-------------+

----------- 2018-05-22 09:22:46 -----------
+--------------+-------------+
|       hashtag|hashtag_count|
+--------------+-------------+
|     #judibola|            1|
|    #Immobile,|            1|
|    #agenbola…|            1|
|      #nigeria|            1|
|      #Icardi,|            1|
|         #Nike|            1|
| 

In [16]:
ssc.stop()

In [22]:
Tokens.show()

AttributeError: 'list' object has no attribute 'show'

In [20]:
type(Tokens)

list